In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

SPREADSHEET_ID = '1AT5IjmCi58VH4HSRrmIHZSNw6v3QNnfTf8QW846neQ0'
RANGE = 'E2:E'

def main():
    creds = None
    if os.path.exists('token.json'):
        creds: Credentials | None = Credentials.from_authorized_user_file('token.json')
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    try:
        service = build('sheets', 'v4', credentials=creds)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE).execute()
        values = result.get('values', [])

        if not values:
            print('No data found.')
            return

        values_np = np.ndarray(values)
        fig, ax = plt.plot()
        ax.hist(values_np, bins=10)

        plt.show()

    except HttpError as err:
        print('Error: {}'.format(err))

if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=904048742741-vnascjlv71u0f0hcjn333hmueqnl8rd4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52942%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets.readonly&state=X92IcdiaslqxPOyfPzBSaho4XEStt8&access_type=offline
